In [1]:
import pandas as pd

In [2]:
training_data_df = pd.read_csv("sales_data_test.csv")
test_data_df = pd.read_csv("sales_data_test.csv")
training_data_df.head()

,critic_rating,is_action,is_exclusive_to_us,is_portable,is_role_playing,is_sequel,is_sports,suitable_for_kids,total_earnings,unit_price
0,3.5,1,1,1,0,1,0,1,247537,59.99
1,2.5,0,0,0,1,1,0,0,73960,59.99
2,3.5,0,0,0,0,1,1,0,82671,59.99
3,4.0,1,1,0,0,1,0,0,137456,39.99
4,2.0,1,0,1,0,1,0,0,89639,59.99


In [3]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

In [4]:
scaled_training = scaler.fit_transform(training_data_df)
scaled_testing = scaler.transform(test_data_df)

In [5]:
print("Note: total_earnings values were scaled by multiplying by {:.10f} and adding {:.6f}".format(scaler.scale_[8], scaler.min_[8]))

Note: total_earnings values were scaled by multiplying by 0.0000042367 and adding -0.153415


In [6]:
pd.DataFrame(data=scaled_training, columns=training_data_df.columns.values).to_csv("sales_data_training_scaled.csv", index=False)

In [7]:
pd.DataFrame(data=scaled_testing, columns=test_data_df.columns.values).to_csv("sales_data_testing_scaled.csv.csv", index=False)

In [8]:
from keras.models import Sequential
from keras.layers import *
import tensorflow as tf

Using TensorFlow backend.


In [9]:
training_data_df = pd.read_csv("sales_data_training_scaled.csv")

X = training_data_df.drop('total_earnings', axis=1).values
Y = training_data_df[['total_earnings']].values

In [10]:
model = Sequential()
model.add(Dense(50, input_dim=9, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss="mean_squared_error", optimizer="adam")

In [11]:
model.fit(X,Y,epochs=500,shuffle=True,verbose=0)

In [12]:
test_data_df = pd.read_csv("sales_data_test_scaled.csv")

In [13]:
X_test = test_data_df.drop('total_earnings', axis=1).values
Y_test = test_data_df[['total_earnings']].values

In [14]:
test_error_rate = model.evaluate(X_test, Y_test, verbose=0)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))

The mean squared error (MSE) for the test data set is: 0.0011083447700366378


In [15]:
X = pd.read_csv("proposed_new_product.csv").values

In [16]:
prediction = model.predict(X)

In [17]:
(prediction[0][0] - scaler.min_[8])/scaler.scale_[8]

262658.527412951

In [18]:
print("Earnings Prediction for Proposed Product - ${}".format((prediction[0][0] - scaler.min_[8])/scaler.scale_[8]))

Earnings Prediction for Proposed Product - $262658.527412951


In [19]:
model.save("trained_model.h5")
print("Model saved to disk.")

Model saved to disk.


In [20]:
from keras.models import load_model
model = load_model('trained_model.h5')

In [21]:
X = pd.read_csv("proposed_new_product.csv").values
prediction = model.predict(X)

print("Earnings Prediction for Proposed Product - ${}".format((prediction[0][0] - scaler.min_[8])/scaler.scale_[8]))

Earnings Prediction for Proposed Product - $262658.527412951
